<a href="https://colab.research.google.com/github/iliza5522/QM22/blob/main/NLP_stakeholders_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm

import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

In [7]:
url='https://www.uscc.gov/sites/default/files/transcripts/Hearing%20Transcript%20-%20January%2025%2C%202018_0.pdf'
raw = parser.from_file(url)

INFO:tika.tika:Retrieving https://www.uscc.gov/sites/default/files/transcripts/Hearing%20Transcript%20-%20January%2025%2C%202018_0.pdf to /tmp/sites-default-files-transcripts-hearing-20transcript-20-20january-2025-2c-202018_0.pdf.


In [8]:
raw['metadata']
if 'dc:title' in raw['metadata']:
    title = raw['metadata']['dc:title']
else:
    title = 'Title not found'

raw['metadata']
if 'dcterms:created' in raw['metadata']:
    date = raw['metadata']['dcterms:created']
else:
    date = 'Date not found'
raw_text=raw['content']

print('Date: ', date)
print('Title: ', title)
print('Word Count: ', len(raw_text))
print('Text:')
raw_text

Date:  2018-02-26T18:21:02Z
Title:  Title not found
Word Count:  624164
Text:


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHEARING ON CHINA\'S BELT AND ROAD INITIATIVE: \nFIVE YEARS LATER \n\nHEARING \n\nBEFORE THE \n\nU.S .-CHINA ECONOMIC AND SECURITY REVIEW COMMISSION\n\nONE HUNDRED FIFTEENTH CONGRESS \nSECOND  SESSION \n\nTHURSDAY, JANUARY 25,  2018 \n\nPrin ted  for  use  of  the   \nUni ted  States-China Economic and Securi t y  Review Commission \n\nAvai lab le  v ia  the  World  Wide Web:  www.uscc .gov \n\nUNITED STATES-CHINA ECONOMIC AND SECURITY REVIEW \nCOMMISSION \n\nWASHINGTON:  2018 \n\nhttp://www.uscc.gov/\n\n\nU.S.-CHINA ECONOMIC AND SECURITY REVIEW COMMISSION\n\nROBIN CLEVELAND, CHAIRMAN \nCAROLYN BARTHOLOMEW, VICE  CHAIRMAN \n\nCommissioners :  \nHON.  CARTE P .  GOODWIN   HON.  JAMES M. TALENT \nDR.  GLENN HUBBARD  HON.  KATHERINE C.  TOBIN  \nHON.  DENNIS C.  SHEA MICHAEL R.  WESSEL  \nHON.  JONATHAN N. STIVERS DR.  LARRY M. WORTZEL  \n\nThe Commission was created on October 30, 2000 by the Floyd D. S

In [9]:
call=pd.DataFrame({'Date':[date],'Title':[title],'Text':[raw_text]})

call['Date']=pd.to_datetime(call['Date'])

call

,Date,Title,Text
0,2018-02-26 18:21:02+00:00,Title not found,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [12]:
import re

def extract_dialogues(text, speakers):
    speaker_patterns = {speaker: re.compile(rf"({speaker}):") for speaker in speakers}
    dialogues = {}
    current_speaker = None
    current_dialogue = ""
    for line in text.split('\n'):
        line = line.strip()
        if not line:
            continue

        # Check if a new speaker has started
        found_speaker = False
        for speaker, pattern in speaker_patterns.items():
            match = pattern.match(line)
            if match:
                if current_speaker:  # Save the previous speaker's dialogue
                    dialogues.setdefault(current_speaker, []).append(current_dialogue.strip())
                current_speaker = speaker  # Start tracking the new speaker
                current_dialogue = line[match.end():].strip()  # Capture their dialogue
                found_speaker = True
                break  # Move to the next line

        # If it's not a new speaker, add to the current dialogue
        if not found_speaker and current_speaker:
            current_dialogue += " " + line  # Append the line to the current speaker's dialogue
            if len(current_dialogue.split()) > 300:  # Check if 300 words exceeded
                dialogues.setdefault(current_speaker, []).append(current_dialogue.strip())
                current_speaker = None
                current_dialogue = ""

    # Add the last dialogue if present
    if current_speaker:
        dialogues.setdefault(current_speaker, []).append(current_dialogue.strip())
    return dialogues

# Your existing code:
speakers = ['HEARING CO-CHAIR SHEA', 'HEARING CO-CHAIR TOBIN', 'MS. ROLLAND', 'MR. HILLMAN', 'MR. PHILLIPS', 'COMMISSIONER TALENT', 'CHAIRMAN CLEVELAND', 'VICE CHAIRMAN BARTHOLOMEW', 'COMMISSIONER STIVERS', 'DR. RATNER', 'DR. KLIMAN', 'MR. SMALL', 'MR. HARRIS', 'DR. EISENMAN']
dialogues = extract_dialogues(raw_text, speakers)

for speaker, dialogues_list in dialogues.items():
    for i, dialogue in enumerate(dialogues_list):
        print(f"{speaker}: {dialogue}")
        print("\n")

HEARING CO-CHAIR TOBIN: Good morning, and welcome to the first hearing of the U.S.- China Economic and Security Review Commission's 2018 Annual Report cycle.  Thank you all for joining us, and those of you online as well. The Commission has been following Xi Jinping's Belt and Road Initiative since its inception in 2013.  Since then, we have watched the initiative rise to the top of Beijing's foreign policy agenda, evolving from a vague blueprint for regional connectivity to a well- resourced, whole-of-government strategy. President Xi's signature foreign policy initiative is many things: an economic stimulus plan; the global face of the "China Dream"; a marketing strategy; an upgrade of Beijing's "going out" policy; a blueprint for economic dominance; and much more. It's difficult to overstate the potential risks and rewards of the Belt and Road Initiative, both for China and for partnering countries. Chinese infrastructure projects have the potential to offload some of the excess ind